In [ ]:
# !git clone https://github.com/ContinualAI/continual-learning-baselines.git

In [6]:
# Upgrade pip first
!python -m pip install --upgrade pip

# Core dependencies
!pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --extra-index-url https://download.pytorch.org/whl/cu118
!pip install avalanche-lib==0.6.0 matplotlib==3.10.5 numpy==1.26.1 tqdm==4.67.1 kornia==0.8.1
!pip install torchmetrics==0.11.4 lightning-utilities==0.10.0 packaging --upgrade

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118


In [5]:
# Requirements
import numpy as np

In [3]:
# Old import
# from avalanche.evaluation import metric_helpers as mh

# New Avalanche 0.6+ way:
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics
from avalanche.training.plugins import EvaluationPlugin

# Example:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=False, epoch=True),
    loss_metrics(minibatch=False, epoch=True)
)

/usr/local/lib/python3.10/dist-packages/avalanche/training/plugins/evaluation.py:94: UserWarning: No loggers specified, metrics will not be logged
  warnings.warn("No loggers specified, metrics will not be logged")


In [4]:
# As a test to see if the plugin works 
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics

# Create a simple evaluation plugin
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=False, epoch=True),
    loss_metrics(minibatch=False, epoch=True)
)

print("Avalanche evaluation plugin created successfully!")

Avalanche evaluation plugin created successfully!


In [5]:
from experiments.split_mnist import synaptic_intelligence_smnist
from avalanche.evaluation import metrics
from avalanche.training.plugins import EvaluationPlugin

In [13]:
"""
import torch

# To calculate memory in MB
def get_model_size_mb(model):
    """Compute memory usage of a PyTorch model in megabytes."""
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()  # bytes
    return param_size / (1024 ** 2)  # convert to MB
"""

In [6]:
# Run SI on Split MNIST
res = synaptic_intelligence_smnist(
    override_args={'epochs': 2}  # faster for CPU
)

# Inspect top-level keys
print("Keys in result:", res.keys())

# If metric values are nested, find them dynamically
accuracy_matrix = []
for key in res:
    if 'Top1_Acc' in key:  # adjust if needed
        accuracy_matrix.append(res[key])

accuracy_matrix = np.array(accuracy_matrix)
print("Accuracy matrix shape:", accuracy_matrix.shape)
print("Accuracy matrix:\n", accuracy_matrix)

# Compute CL metrics 
ACC = accuracy_matrix[-1]
F = np.mean(np.max(accuracy_matrix[:-1]) - accuracy_matrix[-1])  # forgetting

# Approximate FWT
FWT = np.mean(accuracy_matrix[:-1])

# Compute BWT
BWT = np.mean(accuracy_matrix[-1] - accuracy_matrix[:-1])

PSM = ACC / (ACC + abs(F))

# Memory usage (MB)
# memory_mb = get_model_size_mb(si_model)  # use your real model, not res['model']
# print("Memory usage (MB):", memory_mb)

print("ACC:", ACC)
print("F:", F)
print("FWT (approx.):", FWT)
print("BWT:", BWT)
print("PSM:", PSM)
# print("Memory usage (MB):", memory_mb)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:00<00:00, 80409304.39it/s]


Extracting /root/.avalanche/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 2748523.90it/s]


Extracting /root/.avalanche/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



...

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 6125571.95it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw




/usr/local/lib/python3.10/dist-packages/avalanche/training/templates/base.py:468: PositionalArgumentsDeprecatedWarning: Avalanche is transitioning to strategy constructors that accept named (keyword) arguments only. This is done to ensure that there is no confusion regarding the meaning of each argument (strategies can have many arguments). Your are passing 3 positional arguments to the SynapticIntelligence.__init__ method. Consider passing them as names arguments. The ability to pass positional arguments will be removed in the future.
  warnings.warn(error_str, category=PositionalArgumentsDeprecatedWarning)


-- >> Start of training phase << --
100%|██████████| 396/396 [00:28<00:00, 13.68it/s]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9916
100%|██████████| 396/396 [00:29<00:00, 13.57it/s]
Epoch 1 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9979
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 17/17 [00:01<00:00, 15.31it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9991
-- Starting eval on experience 1 (Task 1) from test stream --
100%|██████████| 16/16 [00:01<00:00, 14.35it/s]
> Eval on experience 1 (Task 1) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task001/Exp001 = 0.0000
-- Starting eval on experience 2 (Task 2) from test stream --
100%|██████████| 15/15 [00:00<00:00, 16.54it/s]
> Eval on experience 2 (Task 2) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_st

In [6]:
from experiments.permuted_mnist import synaptic_intelligence_pmnist

In [8]:
# Run SI on Permuted MNIST
res = synaptic_intelligence_pmnist(override_args={'epochs': 2})

# Inspect top-level keys
print("Keys in result:", res.keys())

# If metric values are nested, find them dynamically
accuracy_matrix = []
for key in res:
    if 'Top1_Acc' in key:  # adjust if needed
        accuracy_matrix.append(res[key])

accuracy_matrix = np.array(accuracy_matrix)
print("Accuracy matrix shape:", accuracy_matrix.shape)
print("Accuracy matrix:\n", accuracy_matrix)

# Compute CL metrics 
ACC = accuracy_matrix[-1]
F = np.mean(np.max(accuracy_matrix[:-1]) - accuracy_matrix[-1])  # forgetting

# Approximate FWT
FWT = np.mean(accuracy_matrix[:-1])

# Compute BWT
BWT = np.mean(accuracy_matrix[-1] - accuracy_matrix[:-1])

PSM = ACC / (ACC + abs(F))

# Memory usage (MB)
# memory_mb = get_model_size_mb(si_model)  # use your real model, not res['model']
# print("Memory usage (MB):", memory_mb)

print("ACC:", ACC)
print("F:", F)
print("FWT (approx.):", FWT)
print("BWT:", BWT)
print("PSM:", PSM)
# print("Memory usage (MB):", memory_mb)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:00<00:00, 78111218.03it/s]


Extracting /root/.avalanche/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 2610908.13it/s]


Extracting /root/.avalanche/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



...

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 3603959.28it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw




/usr/local/lib/python3.10/dist-packages/avalanche/training/templates/base.py:468: PositionalArgumentsDeprecatedWarning: Avalanche is transitioning to strategy constructors that accept named (keyword) arguments only. This is done to ensure that there is no confusion regarding the meaning of each argument (strategies can have many arguments). Your are passing 3 positional arguments to the SynapticIntelligence.__init__ method. Consider passing them as names arguments. The ability to pass positional arguments will be removed in the future.
  warnings.warn(error_str, category=PositionalArgumentsDeprecatedWarning)


-- >> Start of training phase << --
100%|██████████| 938/938 [01:22<00:00, 11.41it/s]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8550
100%|██████████| 938/938 [01:19<00:00, 11.86it/s]
Epoch 1 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9185
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:06<00:00, 13.13it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9289
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:06<00:00, 12.74it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.1016
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 79/79 [00:06<00:00, 12.31it/s]
> Eval on experience 2 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_st

In [7]:
# Split CIFAR-100
from experiments.si_split_cifar100 import si_split_cifar100
print(si_split_cifar100)

<function si_split_cifar100 at 0x7186455be170>


In [7]:
# Run SI on Split CIFAR-100
res = si_split_cifar100({'epochs': 2})  # test run
print("Type of res:", type(res))

accuracy_matrix = []

if isinstance(res, dict):
    for key in res:
        if 'Top1_Acc' in key:
            accuracy_matrix.append(res[key])

elif isinstance(res, list):
    if len(res) > 0 and isinstance(res[0], dict):
        # list of dicts
        for r in res:
            for key in r:
                if 'Top1_Acc' in key:
                    accuracy_matrix.append(r[key])
    else:
        # assume it's a list of accuracies already
        accuracy_matrix = res

accuracy_matrix = np.array(accuracy_matrix)
print("Accuracy matrix shape:", accuracy_matrix.shape)
print("Accuracy matrix:\n", accuracy_matrix)

# Compute CL metrics
ACC = accuracy_matrix[-1]
F = np.mean(np.max(accuracy_matrix[:-1]) - accuracy_matrix[-1])
FWT = np.mean(accuracy_matrix[:-1])
BWT = np.mean(accuracy_matrix[-1] - accuracy_matrix[:-1])
PSM = ACC / (ACC + abs(F))

print("ACC:", ACC)
print("F:", F)
print("FWT (approx.):", FWT)
print("BWT:", BWT)
print("PSM:", PSM)

NameError: name 'si_split_cifar100' is not defined

In [8]:
# Split CIFAR-10
from experiments.si_split_cifar10 import si_split_cifar10
print(si_split_cifar10)

<function si_split_cifar10 at 0x7a05d436bd00>


In [12]:
# Run SI on Split CIFAR-10
res = si_split_cifar10({'epochs': 2})  
print("Type of res:", type(res))

accuracy_matrix = []

if isinstance(res, dict):
    for key in res:
        if 'Top1_Acc' in key:
            accuracy_matrix.append(res[key])

elif isinstance(res, list):
    if len(res) > 0 and isinstance(res[0], dict):
        # list of dicts
        for r in res:
            for key in r:
                if 'Top1_Acc' in key:
                    accuracy_matrix.append(r[key])
    else:
        # assume it's a list of accuracies already
        accuracy_matrix = res

accuracy_matrix = np.array(accuracy_matrix)
print("Accuracy matrix shape:", accuracy_matrix.shape)
print("Accuracy matrix:\n", accuracy_matrix)

# Compute CL metrics
ACC = accuracy_matrix[-1]
F = np.mean(np.max(accuracy_matrix[:-1]) - accuracy_matrix[-1])
FWT = np.mean(accuracy_matrix[:-1])
BWT = np.mean(accuracy_matrix[-1] - accuracy_matrix[:-1])
PSM = ACC / (ACC + abs(F))

print("ACC:", ACC)
print("F:", F)
print("FWT (approx.):", FWT)
print("BWT:", BWT)
print("PSM:", PSM)

Files already downloaded and verified
Files already downloaded and verified


UnboundLocalError: local variable 'metrics' referenced before assignment